# Business understanding
Het ziekenhuis Isala wil de zorg voor diabetespatiënten verbeteren. Het gaat hier specifiek om het behandelproces door onnauwkeurigheden in metingen te filteren en hierop te baseren of een behandelplan aangepast moet worden of niet. Visualisaties zullen helpen bij het verkrijgen van inzicht van diverse meetmethoden om vervolgens conclusies te kunnen trekken. Die zijn er nog niet. deze onnauwkeurigheden voor aangegeven als CV. De CV is de variatiecoëfficiënt. Dit is een maat voor de relatieve variabiliteit. Het is de verhouding tussen de standaardafwijking en het gemiddelde (gemiddelde genomen van de spreiding van meetpunten). Hoe kleiner de variatiecoëfficiënt is, hoe minder de gegevens verspreid zijn vanuit het gemiddelde.

Variatiecoëfficiënt = (Standaardafwijking/ Gemiddelde) * 100.


# Data understanding

Binnen dit Jupyter notebook is gewerkt met een aantal bestanden. De databron is een excel bestand genaamd Complete-dataset-FINAL.xlsx. Dit bestand is een samengevoegd bestand van omgezette pdf rapporten. Deze rapporten zijn te vinden via http://www.ngsp.org/CAPdata.asp. Binnen het excel bestand zijn alle meetmethoden en fabrikanten genormaliseerd, en de waarden uitvoerig gecheckt.

De databron heeft de volgende specificaties:
Grootte: 355 kB.
Aantal kolommen: 12.
Kolomnamen: Method name, N (no. labs), Mean, Bias, CV, Sample, Reference value, Year, Source, Type, Manufacturer.
Aantal rijen: 4685.


# Data preparation
Zoals hierboven is beschreven, is de data omgezet van PDF naar Excel formaat en zijn deze samengevoegd tot één bestand. Om deze data te prepareren word er in dit notebook gefilterd op alleen de belangrijke waardes voor het onderzoek. Het gaat hier om de volgende waardes:
- Mean
- Bias
- CV
- Year
- Manufacturer
- Total Error

In [1]:
# importeren van alle functies
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import plotly.express as px
import cufflinks as cf
import numpy as np
from sklearn.cluster import KMeans
import statsmodels.api as sm

# notebook magic
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# inlezen van het databestand
data=pd.read_excel("Complete-dataset-FINAL.xlsx")

### Business / Data understanding

In [2]:
# bekijk de data.
view = data, data.info(), data.describe().transpose(), data.columns;
view

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4684 entries, 0 to 4683
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Method name          4684 non-null   object 
 1   N (no. labs)         4684 non-null   object 
 2   Mean                 4602 non-null   float64
 3   Bias                 4058 non-null   float64
 4   CV                   4584 non-null   float64
 5   Sample               4684 non-null   object 
 6   Reference Value      4656 non-null   float64
 7   Year                 4684 non-null   int64  
 8   Source (CAP/EurA1c)  4684 non-null   object 
 9   Type (Fresh/frozen)  4684 non-null   object 
 10  Manufacturer         4684 non-null   object 
 11  Total Error          4684 non-null   float64
dtypes: float64(5), int64(1), object(6)
memory usage: 439.2+ KB


(                                       Method name N (no. labs)  Mean  Bias  \
 0                        Vitros 5.1 FS. 4600. 5600          183  6.14 -0.16   
 1                        Vitros 5.1 FS. 4600. 5600          183  7.59 -0.01   
 2                        Vitros 5.1 FS. 4600. 5600          183  9.42  0.22   
 3                        Vitros 5.1 FS. 4600. 5600          210  5.42 -0.18   
 4                        Vitros 5.1 FS. 4600. 5600          210  9.34 -0.06   
 ...                                            ...          ...   ...   ...   
 4679  Vitros 5.1 FS/4600/5600/XT7600 Chem. Systems          204  9.08  0.22   
 4680  Vitros 5.1 FS/4600/5600/XT7600 Chem. Systems          204  6.86  0.02   
 4681  Vitros 5.1 FS/4600/5600/XT7600 Chem. Systems          204  9.40  0.01   
 4682  Vitros 5.1 FS/4600/5600/XT7600 Chem. Systems          204  6.14  0.01   
 4683  Vitros 5.1 FS/4600/5600/XT7600 Chem. Systems          204  7.59  0.07   
 
        CV  Sample  Reference Value  Y

#### interpretatie van de dataset
We zien dat er een aantal kolommen zijn die niet de 4684 records hebben die we verwachten. Dit komt omdat er een aantal null- velden zijn. Ook komen er een aantal kolommen voor waar we niets aan hebben, want we hoeven geen methodenamen, samples, Reference Values, N (aantal labs), source of type mee te nemen in dit onderzoek. Hier moet rekening mee gehouden worden in de realisatie en de data preparatiefase.

### Data preparation

In [3]:
# Alleen de kolommen die van toepassing zijn, terug laten komen in de dataset en de values sorteren.
df = data[["Mean","Bias","CV","Year","Manufacturer","Total Error"]];
df = df.sort_values(["Manufacturer"]);

# Groeperen van manufacturer op jaar, en daarna de index setten op manufacturer.
df2 = df.groupby(["Manufacturer","Year"]).mean()
df2 = df2.reset_index()
df2 = df2.set_index('Manufacturer')

# Data gegroepeerd zonder index op de manufacturer.
df3 = df2.reset_index()

### Modeling

#### Beschrijvende modellen

In [4]:
# Tel het aantal jaar dat een merk actief is. Mocht deze minder dan vijf jaar voorkomen, haal die dan uit de dataset.
value_counts = df3["Manufacturer"].value_counts()
to_remove = value_counts[value_counts <= 4].index
filteropjaar = df3[~df3.Manufacturer.isin(to_remove)]
df = filteropjaar

In [5]:
# Laat de manufacturers zien met datapunten verspreid over meer dan vier jaar
counting = df["Manufacturer"].value_counts()
counting

Abbott                       16
Bio-Rad                      16
Tosoh                        16
Roche Diagnostics            16
Vitros                       16
Beckman Coulter              16
Siemens                      14
Trinity Biotech              13
Sebia                         8
Axis-Shield                   7
Roche Diagnostics/Hitachi     7
Alere                         6
Arkray                        6
Menarini                      5
Metrika                       5
Olympus                       5
Not specified                 5
Name: Manufacturer, dtype: int64

In [6]:
# Maak een functie waarin een manufacturer meegegeven kan worden. Dit laat vervolgens een pairplot zien van die manufacturer.
def plotPair(manufacturer):
    tempdf = df[df["Manufacturer"]== manufacturer]
    sns.pairplot(tempdf, hue='Manufacturer', kind='reg', corner=True, plot_kws={'line_kws':{'color':'red'}});

In [7]:
# Voer de hierboven gemaakte functie uit.
interact(plotPair, manufacturer = df['Manufacturer'].unique());

interactive(children=(Dropdown(description='manufacturer', options=('Abbott', 'Alere', 'Arkray', 'Axis-Shield'…

In [8]:
# Realiseren: laat de lijngrafieken zien die bij een specifieke manufacturer horen.
# Maak een functie genaamd lijngrafiek dat gaat plotten op de manufacturer die word opgegeven.
def lijngrafiek(manufacturer):
    tempdf = df3[df3["Manufacturer"]== manufacturer]
    sns.relplot(data=df3, x="Year", y="CV", hue=tempdf.Manufacturer, kind="line").set(title="CV-waarde voor fabrikant {}".format(manufacturer));
    sns.relplot(data=df3, x="Year", y="Bias", hue=tempdf.Manufacturer, kind="line").set(title="Bias-waarde voor fabrikant {}".format(manufacturer));
    sns.relplot(data=df3, x="Year", y="Total Error", hue=tempdf.Manufacturer, kind="line").set(title="Total Error-waarde voor fabrikant {}".format(manufacturer));

In [9]:
# Roep de functie aan.
interact(lijngrafiek, manufacturer = df['Manufacturer'].unique());

interactive(children=(Dropdown(description='manufacturer', options=('Abbott', 'Alere', 'Arkray', 'Axis-Shield'…

### Voorspellende modellen

In [10]:
# Regressievergelijking waarop kan worden voorspeld door middel van een trendlijn.
def regressie(manufacturer):
    
# maak een tijdelijk dataframe aan en laat hier per manufacturer de waardes zien.
    tempdf = df3[df3["Manufacturer"]== manufacturer]
    sns.regplot(data=tempdf, x="Year", y="CV");
    sns.regplot(data=tempdf, x="Year", y="Bias");
    sns.regplot(data=tempdf, x="Year", y="Total Error").set(title="Regressie voor fabrikant {}".format(manufacturer));
    plt.ylabel("Waarde (in procenten)")

In [11]:
# Roep de functie aan.
interact(regressie, manufacturer = df['Manufacturer'].unique());

interactive(children=(Dropdown(description='manufacturer', options=('Abbott', 'Alere', 'Arkray', 'Axis-Shield'…

In [12]:
# Plot een regressie op de ingevoerde manufacturer. Drop alle lege values en laat de waardes Year en CV zien.
def regressie01(manufacturer):
    tempdf = df2[df2["Manufacturer"]== manufacturer]
    X = tempdf["Year"]
    y = tempdf["CV"]

    # Fit en maak de predictions op basis van het model.
    model = sm.OLS(y, X).fit()
    predictions = model.predict(X)

    # Print de statistieken uit.
    # return model.summary()
    return tempdf

# Print het regressiemodel uit. (uitgecomment i.v.m. error.)
# Foutmelding: keyerror op manufacturer. Dit komt omdat er NAN-values aanwezig zijn. Dropna() is hier de oplossing voor.
# De oplossing wordt hieronder weergegeven bij regressie02.
# regressie01("Abbott")

In [13]:
# Plot een regressie op de ingevoerde manufacturer. Drop alle lege values en laat de waardes Year en CV zien.
def regressie02(manufacturer):
    tempdf = data[data["Manufacturer"]== manufacturer].dropna()
    X = tempdf["Year"]
    y = tempdf["CV"]

    # Fit en maak de predictions op basis van het model.
    model = sm.OLS(y, X).fit()
    predictions = model.predict(X)

    # Print de statistieken uit.
    return model.summary()
    # return tempdf

In [14]:
# Laat het regressiemodel zien voor de fabrikant Abbott.
regressie02("Abbott")

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                     CV   R-squared (uncentered):                   0.683
Model:                            OLS   Adj. R-squared (uncentered):              0.682
Method:                 Least Squares   F-statistic:                              596.3
Date:                Thu, 02 Jun 2022   Prob (F-statistic):                    4.93e-71
Time:                        23:06:53   Log-Likelihood:                         -605.57
No. Observations:                 278   AIC:                                      1213.
Df Residuals:                     277   BIC:                                      1217.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Year           0.0016   6.37e-05     24.420      0.000       0.001       0.002
==============================================================================
Omnibus:                       41.030   Durbin-Watson:                   0.496
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               55.151
Skew:                           1.076   Prob(JB):                     1.06e-12
Kurtosis:                       3.360   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Interpretatie van het model
met een R-squared waarde van 0.68 is dit model niet erg betrouwbaar voor het voorspellen van de CV waarden in de toekomst voor Abbott. Het ligt namelijk onder de 70%.
Daarentegen is de P>|t| value wel onder de 0.05. Dit betekent dat er met zekerheid gezegd kan worden dat er een relatie is tussen Year en CV.

In [15]:
# Groepeer op fabrikant en laat de fabrikanten zien met meer dan 100 datapunten
manufacturer = data.dropna().groupby('Manufacturer')
AantalDatapunten = manufacturer.size().sort_values(ascending=False)
AantalDatapunten = AantalDatapunten[AantalDatapunten > 100]

# Make een nieuw dataframe met de fabrikanten die aan de vorige eis voldoen
df_manufacturer = data[data['Manufacturer'].isin(AantalDatapunten.index)]

# Print het dataframe uit ter controle
df_manufacturer

,Method name,N (no. labs),Mean,Bias,CV,Sample,Reference Value,Year,Source (CAP/EurA1c),Type (Fresh/frozen),Manufacturer,Total Error
0,Vitros 5.1 FS. 4600. 5600,183,6.14,-0.16,2.3,GH2-04,6.30,2011,CAP,Fresh,Vitros,4.348
1,Vitros 5.1 FS. 4600. 5600,183,7.59,-0.01,2.5,GH2-05,7.60,2011,CAP,Fresh,Vitros,4.890
2,Vitros 5.1 FS. 4600. 5600,183,9.42,0.22,3.0,GH2-06,9.20,2011,CAP,Fresh,Vitros,6.100
3,Vitros 5.1 FS. 4600. 5600,210,5.42,-0.18,2.3,GH2-01,5.60,2012,CAP,Fresh,Vitros,4.328
4,Vitros 5.1 FS. 4600. 5600,210,9.34,-0.06,2.6,GH2-02,9.40,2012,CAP,Fresh,Vitros,5.036
...,...,...,...,...,...,...,...,...,...,...,...,...
4679,Vitros 5.1 FS/4600/5600/XT7600 Chem. Systems,204,9.08,0.22,3.1,GH5-01,8.86,2021,CAP,Fresh,Vitros,6.296
4680,Vitros 5.1 FS/4600/5600/XT7600 Chem. Systems,204,6.86,0.02,2.6,GH5-02,6.84,2021,CAP,Fresh,Vitros,5.116
4681,Vitros 5.1 FS/4600/5600/XT7600 Chem. Systems,204,9.40,0.01,3.4,GH5-03,9.39,2021,CAP,Fresh,Vitros,6.674
4682,Vitros 5.1 FS/4600/5600/XT7600 Chem. Systems,204,6.14,0.01,2.5,GH5-04,6.13,2021,CAP,Fresh,Vitros,4.910


### Voorspellend model
Hieronder wordt het voorspellende model getoond. Dit wordt gedaan per manufacturer. 
De CV waarde van 2022 word hiermee voorspeld.

In [16]:
# definieer de functie 'maakModel' dat de manufacturer als input heeft.
def maakModel(manufacturer):
    # voorbereiding op voorspellend model met een tijdelijk dataframe.
    tempdf = df_manufacturer[df_manufacturer["Manufacturer"]== manufacturer].dropna()
    # definieer een X en y waarde.
    X = tempdf["Year"]
    y = tempdf["CV"]

    # maak het voorspellende OLS model met daarop predictions in een dataframe.
    model = sm.OLS(y, X).fit()
    predictions = model.predict(X)
    
    # selecteer de kolommen met daarin de values waar op gepredict moet worden.
    preddf = pd.DataFrame(predictions, columns=['CV'])
    preddf['Year'] = "2022"
    preddf = preddf.groupby('Year').mean().reset_index()
    
    # voeg alle data samen als een compleet dataframe.
    complete = pd.concat([tempdf,preddf])
    complete['Year'] = complete['Year'].astype(str).astype(int)
    complete = complete.reset_index(drop=True)
    
    # Laat de grafiek zien met daarin de voorspelling, waarbij de voorspelling de kleur rood heeft.
    sns.lineplot(x = 'Year', y = 'CV', data = complete)
    sns.lineplot(x = 'Year', y = 'CV', data = complete[(complete['Year'] >= 2021) & (complete['Year'] <= 2022)], color= 'red').set(title='CV-waarde over de jaren heen van fabrikant {}'.format(manufacturer));
    plt.legend(labels=["Bekende data","Voorspelling"])
    # Laat de statistieken zien van het model.
    return model.summary()

In [17]:
# Roep de functie aan.
interact(maakModel, manufacturer=df_manufacturer['Manufacturer'].unique());

interactive(children=(Dropdown(description='manufacturer', options=('Vitros', 'Roche Diagnostics', 'Abbott', '…

### Interpretatie van model
Met een R-squared waarde van 0.962 voor Vitros is er een zeer sterk, bijna perfect verband tussen de CV en de jaren. De P>|t| waarde is onder de 0.05, dit betekent dat er met zekerheid gezegd kan worden dat er een relatie is tussen Year en CV. Voor Vitros is dit goed nieuws, want de voorspelling is vrijwel accuraat.

Voor de fabrikant Abbott is dit helaas niet het geval, aangezien er een grote dip is geweest in 2021 waardoor het model de voorspelde waarde niet goed kan berekenen. Met een R-squared waarde van 0.683 wordt er aangegeven dat het model niet goed een verband aangeeft tussen CV en Year. De P waarde is daarentegen wel goed; helaas kan er door de R-squared value niet gezegd worden dat de voorspelling alsnog betrouwbaar is.

# Evaluatie
De product owner vond het belangrijk om inzicht te krijgen in hoe de CV, Bias, en Total Error waardes het over de jaren heen deden per fabrikant. Om dit te visualiseren zijn deze waarden meegenomen in het onderzoek, en deze gevisualiseerd. Helaas zijn de waardes die wij hebben samengevoegd (CAP-data en EURA1C data) niet juist berekend. Dit was geen struikelblok aangezien wij dit gescheiden hadden in een kolom genaamd 'source'. Alle waardes zijn desondanks WEL meegenomen in dit onderzoek in verband met de tijd en zijn dus NIET gescheiden van elkaar in dit Jupyter notebook.

Er missen veel datapunten in het onderzoek, en daardoor is het lastig om te gaan voorspellen hoe de fabrikanten het over de jaren heen doen. Enkele fabrikanten zijn maar voor 1 a 2 jaar bezig met het produceren van een product. Dit geeft een vertekend beeld en hierdoor zijn wij helaas gedwongen om deze fabrikanten niet mee te nemen in het onderzoek.


### Conclusie
Bij vrijwel alle manufacturers daalt de cv waarde heel veel. Hier is het voorspellingsmodel niet op voorbereid, waardoor het voor het jaar 2022 erg hoog uitvalt. Dit heeft veel invloed op de voorspelling. Het kan daarom interessant zijn om onderzoek te doen naar waarom dit het geval is. De beschrijvende modellen zijn goed mee te nemen in een vervolgonderzoek. Zij geven een erg goed en duidelijk inzicht over hoe een fabrikant zich heeft ontwikkeld over de jaren heen. Dit is een belangrijk punt voor onze product owner.


### Advies
Het advies aan de product owner is om het voorspellend model niet te gaan gebruiken in een productieomgeving om waardes te gaan voorspellen. Ik adviseer de product owner om meer data beschikbaar te stellen voor het onderzoek naar zowel meetmethodes als fabrikanten. Dit advies is vastgesteld op basis van de CV-predictie (voorspellend model).

Het tweede advies aan de product owner is om onderzoek te doen naar de slechter presterende fabrikanten, en de dip in het jaar 2021. Dit gaat helaas buiten onze scope om en is daarom niet meegenomen in het onderzoek.

Ten slotte adviseren wij om deze bevindingen door te spelen aan een projectgroep die dit als vervolgopdracht op kan pakken. In verband met korte werkdagen, een delay met de stakeholder, en een wijziging in de aanpak van waardeberekeningen kwam onze projectgroep in tijdsnood te zitten en hebben wij ons helaas moeten beperken tot het visualiseren van enkel de CV. Dit omdat de Bias en de Total Error niet meer kloppen volgens de huidige berekening.